<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-Hybrid_Multi_Method

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the `Hybrid_Multi_Method` for selecting different conservative-to-primitive routines at runtime.

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#introduction): **Introduction**
1. [Step 2](#driver_conserv_to_prims): **`driver_conserv_to_prims.C`**
    1. [Step 2.a](#adm_to_bssn__enforcing_detgammabar_equal_one): *Converting ADM quantities to BSSN quantities and enforcing $\bar\gamma=1$*
    1. [Step 2.b](#equatorial_symmetry): *Applying equatorial symmetry*
    1. [Step 2.c](#variable_setup): *Setting up the variables needed by `HARM`*
        1. [Step 2.c.i](#variable_setup__bssn): BSSN quantities
        1. [Step 2.c.ii](#variable_setup__prims): Primitives
        1. [Step 2.c.iii](#variable_setup__conservs): Conservatives
        1. [Step 2.c.iv](#variable_setup__lapse_and_psi): Lapse function and conformal factor
        1. [Step 2.c.v](#variable_setup__phys_metric): Physical spatial metric
        1. [Step 2.c.vi](#variable_setup__betadown_and_beta2): $\beta_{i}$ and $\beta^{2} \equiv \beta_{i}\beta^{i}$
        1. [Step 2.c.vii](#variable_setup__adm_4metric): The ADM 4-metric, $g_{\mu\nu}$, and its inverse, $g^{\mu\nu}$
        1. [Step 2.c.viii](#variable_setup__temp_conservs): Temporary storage for current values of the conservative variables
    1. [Step 2.d](#conserv_to_prim__driver): *Determining the primitives variables from the conservatives variables*
    1. [Step 2.e](#enforce_limits_on_primitives_and_recompute_conservs): *Enforcing physical limits on primitives and recomputing the conservatives variables*
    1. [Step 2.f](#updating_conservs_and_prims_gfs): *Updating conservative and primitive gridfunctions*
    1. [Step 2.g](#diagnostics_and_debugging_tools): *Diagnostics and debugging tools*
1. [Step 3](#harm_primitives_lowlevel): **`harm_primitives_lowlevel.C`**
    1. [Step 3.a](#variables_needed_by_harm): *Setting up the variables needed by `HARM`*
        1. [Step 3.a.i](#variables_needed_by_harm__detg): ${\rm detg}$
        1. [Step 3.a.ii](#variables_needed_by_harm__bi_harm): $B^{i}_{\rm HARM}$
        1. [Step 3.a.iii](#variables_needed_by_harm__init_rhob_pressure_vi): Initializing $\rho_{b}$, $P$, and $v^{i}$
        1. [Step 3.a.iv](#variables_needed_by_harm__original_conserv): Storing the original values of the conservative variables
        1. [Step 3.a.v](#variables_needed_by_harm__guessing_rhob_pressure_vi): Guessing $\rho_{b}$, $P$, and $v^{i}$
        1. [Step 3.a.vi](#variables_needed_by_harm__conservs): Writing $\boldsymbol{C}_{\rm HARM}$ in terms of $\boldsymbol{C}_{\rm IGM}$
        1. [Step 3.a.vii](#variables_needed_by_harm__prims): Writing $\boldsymbol{P}_{\rm HARM}$ in terms of $\boldsymbol{P}_{\rm IGM}$
    1. [Step 3.b](#calling_harm_conservs_to_prims_solver): *Calling the `HARM` conservative-to-primitive solver*
    1. [Step 3.c](#font_fix) *Applying the Font *et al.* fix, if the inversion fails*
    1. [Step 3.d](#compute_utconi) *Compute $\tilde{u}^{i}$*
    1. [Step 3.e](#limiting_velocities) *Limiting velocities*
    1. [Step 3.f](#primitives) *Setting the primitives*
1. [Step 4](#font_fix_hybrid_eos): **`font_fix_hybrid_EOS.C`**
    1. [Step 4.a](#font_fix_hybrid_eos__basic_quantities): Computing the basic quantities needed by the algorithm
    1. [Step 4.b](#font_fix_hybrid_eos__sdots): Basic check: looking at $\tilde{S}^{2}$
    1. [Step 4.c](#font_fix_hybrid_eos__initial_guesses): Initial guesses for $W$, $S_{{\rm fluid}}^{2}$, and $\rho$
    1. [Step 4.d](#font_fix_hybrid_eos__main_loop): The main loop
    1. [Step 4.e](#font_fix_hybrid_eos__outputs): Output $\rho_{b}$ and $u_{i}$
1. [Step 5](#harm_primitives_headers): **`harm_primitives_headers.h`**
1. [Step 6](#code_validation): **Code validation**
    1. [Step 6.a](#driver_conserv_to_prims_validation): *`driver_conserv_to_prims.C`*
    1. [Step 6.b](#harm_primitives_lowlevel_validation): *`harm_primitives_lowlevel.C`*
    1. [Step 6.c](#font_fix_gamma_law_validation): *`font_fix_gamma_law.C`*
    1. [Step 6.d](#harm_primitives_headers_validation): *`harm_primitives_headers.h`*
1. [Step 7](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

GRHayL provides a variety of conservative-to-primitive (C2P) algorithms. These can be directly used, but the `Hybrid_Multi_Method` function allows for switching C2P methods using runtime parameters. This function also allows for up to 3 C2P methods to serve as backup methods if the preceeding methods fail.

<a id='c2p_routines'></a>

# Step 2: Calling Con2Prim routines \[Back to [top](#toc)\]
$$\label{c2p_routines}$$

First, let's look at the argument list which is shared by all conservative-to-primitive routines:

```
int Hybrid_Multi_Method(
      const grhayl_parameters *restrict params,
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      const conservative_quantities *restrict cons,
      const primitive_quantities *restrict prims,
      primitive_quantities *restrict prims_guess,
      con2prim_diagnostics *restrict diagnostics)
```
The first two arguments appear in most GRHayL functions and contain information on the simulation and equation-of-state parameters. The next struct contains (pointwise) information on the metric. Whenever packing the metric struct, we recommend using the function `GRHayL_enforce_detgtij_and_initialize_metric`, which also enforces that the determinant of the conformal metric equals one. GRHayL functions may behave unpredictably with abnormal metrics, as they have not been tested as rigorously outside the expected input ranges. The conservative_quantities input contains the **undensitized** conservative variables. Since evolving densitized variables is quite common, we provide the `undensitize_conservatives` function which takes a struct containing densitized variables and packs a new struct with undensitized quantities to serve as input for this function. The final input is the primitive_quantities struct `prims`, which must at least contain the $B^i$ variables.

The outputted primitives are packed into the primitive_quantities struct `prims_guess`, which doubles as the initial guess for the conservative-to-primitive routine. If the parameter `params->calc_prim_guess` is true, then `prims_guess` is set by `guess_primitives`. Otherwise, the initial guess is set to whatever was provided via the `prims` input. The $B^i$ in `prims` are always used to set the $B^i$ in `prims_guess`.

These arguments are identical for all the C2P routines, so switching routines is very easy even if they are being called individually and not by using the `Hybrid_Multi_Method` function.

<a id='Hybrid_Multi_Method'></a>

## Step 3.a: The `Hybrid_Multi_Method` Function \[Back to [top](#toc)\]
$$\label{Hybrid_Multi_Method}$$

The `Hybrid_Multi_Method` function has the following structure
```
  set B^i in prims_guess
  set other prims_guess quantities from either prims struct or guess_primitives()
  call params->main_routine C2P function

  if(C2p failed and backup 1 is set)
    set other prims_guess quantities from either prims struct or guess_primitives()
    call params->backup_routine[0] C2P function

    if(C2p failed and backup 2 is set)
      set other prims_guess quantities from either prims struct or guess_primitives()
      call params->backup_routine[1] C2P function

      if(C2p failed and backup 3 is set)
        set other prims_guess quantities from either prims struct or guess_primitives()
        call params->backup_routine[2] C2P function
```
If more backups are desired, the code inside `Hybrid_Multi_Method` can be copied directly into the function calling this one with as many tiered if statements as desired. The parameter in the grhayl_parameters struct only allows for 3 values, so any additional options will have to be handled in-house by the user code.




<a id='Hybrid_Select_Method'></a>

## Step 2.b: The `Hybrid_Select_Method` Function \[Back to [top](#toc)\]
$$\label{Hybrid_Select_Method}$$

The function which calls the C2P routines is `Hybrid_Select_Method`. It differs from all other C2P routines by a single argument
```
int Hybrid_Select_Method(const grhayl_parameters *restrict params,
                         const eos_parameters *restrict eos,
                         const int c2p_key,
                         const metric_quantities *restrict metric,
                         const conservative_quantities *restrict cons,
                         primitive_quantities *restrict prims,
                         con2prim_diagnostics *restrict diagnostics )
```
This function simply consists of a switch statement that calls the C2P routine matching the additional argument `c2p_key`.

<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__driver_conserv_to_prims.pdf](Tutorial-IllinoisGRMHD__driver_conserv_to_prims.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [41]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__the_conservative_to_primitive_algorithm.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__the_conservative_to_primitive_algorithm.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__the_conservative_to_primitive_algorithm.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__the_conservative_to_primitive_algorithm.tex
!rm -f Tut*.out Tut*.aux Tut*.log